In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from keras.layers import BatchNormalization, Lambda, Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from keras.layers.merge import Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action = 'ignore', category= FutureWarning)
%matplotlib inline

In [9]:
def mycolournet():
    input_image = Input(shape=(56,56,3))
    layer1 = MaxPooling2D(pool_size=(1,1),strides=(1,1),input_shape=(56,56,3))(input_image)
    
    blueinput =  Lambda(lambda x : x[:,:,:,0:1])(layer1)
    greeninput =  Lambda(lambda x : x[:,:,:,1:2])(layer1)
    redinput =  Lambda(lambda x : x[:,:,:,:2])(layer1)
    
    
    
    blueconv1 = Convolution2D(filters=16,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(blueinput)
    blueconv1 = BatchNormalization()(blueconv1)
    blueconv1 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(blueconv1)
    
    greenconv1 = Convolution2D(filters=16,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(greeninput)
    greenconv1 = BatchNormalization()(greenconv1)
    greenconv1 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(greenconv1)
    
    redconv1 = Convolution2D(filters=16,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(redinput)
    redconv1 = BatchNormalization()(redconv1)
    redconv1 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(redconv1)
    
    blueconv2 = Convolution2D(filters=32,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(blueconv1)
    blueconv2 = BatchNormalization()(blueconv2)
    blueconv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(blueconv2)
    
    greenconv2 = Convolution2D(filters=32,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(greenconv1)
    greenconv2 = BatchNormalization()(greenconv2)
    greenconv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(greenconv2)
    
    redconv2 = Convolution2D(filters=32,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(redconv1)
    redconv2 = BatchNormalization()(redconv2)
    redconv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(redconv2)
    
    blueflatten = Flatten()(blueconv2)
    greenflatten = Flatten()(greenconv2)
    redflatten = Flatten()(redconv2)
    
    blueFC_1 = Dense(units=200, activation='relu')(blueflatten)
    blueFC_1 = Dropout(0.6)(blueFC_1)
    blueFC_2 = Dense(units=1, activation='relu')(blueFC_1)
    
    greenFC_1 = Dense(units=200, activation='relu')(greenflatten)
    greenFC_1 = Dropout(0.6)(greenFC_1)
    greenFC_2 = Dense(units=1, activation='relu')(greenFC_1)
    
    redFC_1 = Dense(units=200, activation='relu')(redflatten)
    redFC_1 = Dropout(0.6)(redFC_1)
    redFC_2 = Dense(units=1, activation='relu')(redFC_1)
    
    concatoutput = Concatenate()([blueFC_2, greenFC_2,redFC_2])
    
    cpn1 = Dense(units = 100, activation = 'relu')(concatoutput)
    cpn1=  Dense(units = 20 , activation = 'relu')(cpn1)
    cpn1=  Dense(units = 2 , activation = 'relu')(cpn1)
    
    cpn2_1 = Dense(units = 40, activation = 'relu')(cpn1)
    cpn2_1 = Dense(units = 100, activation = 'relu')(cpn2_1)
    cpn2_1 = Dense(units = 4, activation = 'relu')(cpn2_1)  #red, yellow,orange,pink
    
    
    
    cpn2_2 = Dense(units = 40, activation = 'relu')(cpn1)
    cpn2_2 = Dense(units = 100, activation = 'relu')(cpn2_2)
    cpn2_2 = Dense(units = 3, activation = 'relu')(cpn2_2)  #blue, purple, green
    
    finalconcat =Concatenate()([cpn2_1, cpn2_2])
    output= Activation(activation = 'softmax')(finalconcat)
    
    model = Model(inputs=input_image,outputs=output)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    # sgd = SGD(lr=0.01, momentum=0.9, decay=0.0005, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

    
    

In [10]:
model = mycolournet()

In [11]:
train_path = 'data/colors/train'
valid_path = 'data/colors/valid'
test_path = 'data/colors/test'


In [12]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(56,56), classes=['red','yellow','orange', 'pink', 'blue', 'purple','green'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(56,56), classes=['red','yellow','orange', 'pink', 'blue', 'purple','green' ], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(56,56), classes=['red','yellow','orange', 'pink', 'blue', 'purple','green' ], batch_size=10, shuffle=False)

Found 1054 images belonging to 7 classes.
Found 72 images belonging to 7 classes.
Found 2 images belonging to 7 classes.


In [6]:
model.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=200,
    verbose=2)    

Epoch 1/200


C:\Users\Debjyoty\anaconda3\lib\site-packages\PIL\Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


94/94 - 27s - loss: 1.9444 - accuracy: 0.1779 - val_loss: 1.9524 - val_accuracy: 0.0833
Epoch 2/200
94/94 - 11s - loss: 1.9364 - accuracy: 0.2133 - val_loss: 1.9582 - val_accuracy: 0.1111
Epoch 3/200
94/94 - 11s - loss: 1.8961 - accuracy: 0.2744 - val_loss: 2.0039 - val_accuracy: 0.0833
Epoch 4/200
94/94 - 10s - loss: 1.7996 - accuracy: 0.3033 - val_loss: 1.8715 - val_accuracy: 0.1528
Epoch 5/200
94/94 - 11s - loss: 1.7675 - accuracy: 0.3205 - val_loss: 1.9049 - val_accuracy: 0.1528
Epoch 6/200
94/94 - 11s - loss: 1.7359 - accuracy: 0.3355 - val_loss: 1.9172 - val_accuracy: 0.1667
Epoch 7/200
94/94 - 12s - loss: 1.7233 - accuracy: 0.3258 - val_loss: 2.0056 - val_accuracy: 0.1528
Epoch 8/200
94/94 - 11s - loss: 1.7237 - accuracy: 0.3333 - val_loss: 1.9364 - val_accuracy: 0.1667
Epoch 9/200
94/94 - 12s - loss: 1.6975 - accuracy: 0.3398 - val_loss: 1.8554 - val_accuracy: 0.1667
Epoch 10/200
94/94 - 14s - loss: 1.6920 - accuracy: 0.3355 - val_loss: 1.9451 - val_accuracy: 0.1667
Epoch 11/20

Epoch 83/200
94/94 - 15s - loss: 0.7687 - accuracy: 0.7749 - val_loss: 1.4885 - val_accuracy: 0.5833
Epoch 84/200
94/94 - 13s - loss: 0.7513 - accuracy: 0.7878 - val_loss: 1.7060 - val_accuracy: 0.4861
Epoch 85/200
94/94 - 14s - loss: 0.7707 - accuracy: 0.7728 - val_loss: 1.5997 - val_accuracy: 0.5694
Epoch 86/200
94/94 - 13s - loss: 0.7464 - accuracy: 0.7749 - val_loss: 1.5545 - val_accuracy: 0.5556
Epoch 87/200
94/94 - 13s - loss: 0.8291 - accuracy: 0.7471 - val_loss: 1.5079 - val_accuracy: 0.5278
Epoch 88/200
94/94 - 13s - loss: 0.7684 - accuracy: 0.7717 - val_loss: 1.5788 - val_accuracy: 0.5278
Epoch 89/200
94/94 - 15s - loss: 0.8098 - accuracy: 0.7685 - val_loss: 1.5170 - val_accuracy: 0.5833
Epoch 90/200
94/94 - 13s - loss: 0.7326 - accuracy: 0.7889 - val_loss: 1.7493 - val_accuracy: 0.4861
Epoch 91/200
94/94 - 13s - loss: 0.7702 - accuracy: 0.7760 - val_loss: 1.5590 - val_accuracy: 0.5278
Epoch 92/200
94/94 - 13s - loss: 0.7244 - accuracy: 0.7835 - val_loss: 1.5570 - val_accurac

KeyboardInterrupt: 

In [13]:
model.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=200,
    verbose=2)    

Epoch 1/200


C:\Users\Debjyoty\anaconda3\lib\site-packages\PIL\Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


106/106 - 14s - loss: 1.9446 - accuracy: 0.1784 - val_loss: 1.9437 - val_accuracy: 0.1667
Epoch 2/200
106/106 - 14s - loss: 1.9421 - accuracy: 0.1812 - val_loss: 1.9421 - val_accuracy: 0.1667
Epoch 3/200
106/106 - 14s - loss: 1.9409 - accuracy: 0.1812 - val_loss: 1.9399 - val_accuracy: 0.1667
Epoch 4/200
106/106 - 14s - loss: 1.9391 - accuracy: 0.1812 - val_loss: 1.9390 - val_accuracy: 0.1667
Epoch 5/200
106/106 - 15s - loss: 1.9371 - accuracy: 0.1812 - val_loss: 1.9368 - val_accuracy: 0.1667
Epoch 6/200
106/106 - 14s - loss: 1.9295 - accuracy: 0.1812 - val_loss: 1.9310 - val_accuracy: 0.1667
Epoch 7/200
106/106 - 14s - loss: 1.9163 - accuracy: 0.1812 - val_loss: 1.9107 - val_accuracy: 0.1667
Epoch 8/200
106/106 - 14s - loss: 1.8709 - accuracy: 0.2685 - val_loss: 1.8271 - val_accuracy: 0.2778
Epoch 9/200
106/106 - 14s - loss: 1.7830 - accuracy: 0.3340 - val_loss: 1.7688 - val_accuracy: 0.2639
Epoch 10/200
106/106 - 14s - loss: 1.7159 - accuracy: 0.3643 - val_loss: 1.6591 - val_accuracy

Epoch 81/200
106/106 - 14s - loss: 1.1738 - accuracy: 0.5237 - val_loss: 1.5804 - val_accuracy: 0.2917
Epoch 82/200
106/106 - 14s - loss: 1.1624 - accuracy: 0.5512 - val_loss: 1.5423 - val_accuracy: 0.3611
Epoch 83/200
106/106 - 14s - loss: 1.1308 - accuracy: 0.5408 - val_loss: 1.5429 - val_accuracy: 0.3333
Epoch 84/200
106/106 - 14s - loss: 1.0583 - accuracy: 0.5712 - val_loss: 1.6587 - val_accuracy: 0.3472
Epoch 85/200
106/106 - 14s - loss: 1.0401 - accuracy: 0.5797 - val_loss: 1.7382 - val_accuracy: 0.3194
Epoch 86/200
106/106 - 14s - loss: 1.1104 - accuracy: 0.5427 - val_loss: 1.6807 - val_accuracy: 0.3333
Epoch 87/200
106/106 - 14s - loss: 1.0519 - accuracy: 0.5778 - val_loss: 1.6161 - val_accuracy: 0.3194
Epoch 88/200
106/106 - 14s - loss: 1.0583 - accuracy: 0.5797 - val_loss: 1.7662 - val_accuracy: 0.2778
Epoch 89/200
106/106 - 14s - loss: 1.0492 - accuracy: 0.5702 - val_loss: 1.7363 - val_accuracy: 0.2639
Epoch 90/200
106/106 - 14s - loss: 1.1699 - accuracy: 0.5389 - val_loss: 

Epoch 160/200
106/106 - 14s - loss: 1.4895 - accuracy: 0.4184 - val_loss: 1.8839 - val_accuracy: 0.2222
Epoch 161/200
106/106 - 14s - loss: 1.4974 - accuracy: 0.3928 - val_loss: 1.8339 - val_accuracy: 0.2639
Epoch 162/200
106/106 - 14s - loss: 1.4725 - accuracy: 0.4108 - val_loss: 1.8159 - val_accuracy: 0.2500
Epoch 163/200
106/106 - 14s - loss: 1.3710 - accuracy: 0.4469 - val_loss: 1.7917 - val_accuracy: 0.3194
Epoch 164/200
106/106 - 14s - loss: 1.3392 - accuracy: 0.4829 - val_loss: 1.7903 - val_accuracy: 0.2917
Epoch 165/200
106/106 - 14s - loss: 1.3535 - accuracy: 0.4687 - val_loss: 1.8711 - val_accuracy: 0.2778
Epoch 166/200
106/106 - 14s - loss: 1.4410 - accuracy: 0.4345 - val_loss: 1.8961 - val_accuracy: 0.2361
Epoch 167/200
106/106 - 14s - loss: 1.3855 - accuracy: 0.4402 - val_loss: 1.9206 - val_accuracy: 0.2361
Epoch 168/200
106/106 - 14s - loss: 1.3655 - accuracy: 0.4829 - val_loss: 1.7799 - val_accuracy: 0.2778
Epoch 169/200
106/106 - 14s - loss: 1.2914 - accuracy: 0.5066 - 

KeyboardInterrupt: 